### Neural Network Implementierung

Dieses Modell führt sowohl die Single- als auch die Multi-Label Klassifikation in einem Neuronalen Netz durch. Darin hat jedes Label einen eigenen Ein- und Ausgang.

In [12]:
import numpy as np
import pickle
import json
import pandas as pd
import re
from keras.models import load_model
import tensorflow as tf

Anzeige der Tensorflow Version zur Kontrolle. Bei Abweichenden Versionen kann die Klassifikation fehlschlagen

In [2]:
tf.__version__

'2.15.0'

Laden der trainierten Modelle.

In [6]:
VERSION = "5g2"

# Pfad zum gespeicherten Modell und den Komponenten
model_directory = f'./model_files'

# Modell laden
model_filename = f'{model_directory}/neural_network-v{VERSION}.h5'
model = load_model(model_filename)

# TfidfVectorizer laden
with open(f'{model_directory}/tfidf_vectorizer-v{VERSION}.pkl', 'rb') as file:
    tfidf_vectorizer = pickle.load(file)

# MultiLabelBinarizer-Instanzen laden
with open(f'{model_directory}/required_encoder-v{VERSION}.pkl', 'rb') as file:
    mlb_required = pickle.load(file)

with open(f'{model_directory}/select_encoder-v{VERSION}.pkl', 'rb') as file:
    mlb_select = pickle.load(file)

# Fügen Sie ähnliche Blöcke hinzu, um andere Encoder und Binarizer zu laden
with open(f'{model_directory}/expand_encoder-v{VERSION}.pkl', 'rb') as file:
    mlb_expand = pickle.load(file)

with open(f'{model_directory}/expand_select_encoder-v{VERSION}.pkl', 'rb') as file:
    mlb_expand_select = pickle.load(file)

with open(f'{model_directory}/filter_encoder-v{VERSION}.pkl', 'rb') as file:
    mlb_filter = pickle.load(file)

# LabelEncoder-Instanzen laden
with open(f'{model_directory}/method_encoder-v{VERSION}.pkl', 'rb') as file:
    method_encoder = pickle.load(file)

with open(f'{model_directory}/endpoint_encoder-v{VERSION}.pkl', 'rb') as file:
    endpoint_encoder = pickle.load(file)

with open(f'{model_directory}/presentation_encoder-v{VERSION}.pkl', 'rb') as file:
    presentation_encoder = pickle.load(file)

Funktion für die Vorverarbeitung des Prompts

In [8]:
# Funktion zur Bereinigung des Textes
def clean_text(text):
    # Überprüfen auf NaN-Werte
    if pd.isna(text):
        return text
    # Entfernen von Sonderzeichen und Ziffern, behalten von Leerzeichen und Buchstaben
    text = re.sub(r'[^A-Za-zäöüßÄÖÜ\s]', '', text)
    # Umwandlung in Kleinbuchstaben
    text = text.lower()
    # Ersetzen mehrfacher Leerzeichen durch ein einzelnes Leerzeichen
    text = re.sub(r'\s+', ' ', text)
    # Entfernen von Leerzeichen am Anfang und Ende
    text = text.strip()
    return text

Durchführung der Vorhersage für die Labels

In [9]:
USER_PROMPT = "Hat der Auftrag 4711 einen Wareneingang?"
pre_user_prompt = clean_text(USER_PROMPT)

# Vektorisierung des gereinigten Textes
vectorized_test_text = tfidf_vectorizer.transform([pre_user_prompt]).toarray()

# Modellvorhersage
y_pred = model.predict(vectorized_test_text)

1/1 [==============================] - 0s 284ms/step


Vorhergesagte Werte in Klartext umwandeln

In [10]:
# Single-Label-Vorhersagen interpretieren
predicted_method = method_encoder.inverse_transform(np.argmax(y_pred[0], axis=1).reshape(-1))[0]
predicted_endpoint = endpoint_encoder.inverse_transform(np.argmax(y_pred[1], axis=1).reshape(-1))[0]
predicted_presentation = presentation_encoder.inverse_transform(np.argmax(y_pred[2], axis=1).reshape(-1))[0]

threshold = 0.5
# Multi-Label-Vorhersagen interpretieren
predicted_required = mlb_required.inverse_transform((y_pred[3] > threshold).astype(int))[0]
predicted_select = mlb_select.inverse_transform((y_pred[4] > threshold).astype(int))[0]
predicted_filter = mlb_filter.inverse_transform((y_pred[5] > threshold).astype(int))[0]
predicted_expand = mlb_expand.inverse_transform((y_pred[6] > threshold).astype(int))[0]
predicted_expand_select = mlb_expand_select.inverse_transform((y_pred[7] > threshold).astype(int))[0]

Ausgabe der vorhergesagten Werte in einer JSON

In [15]:
# Umwandlung der Vorhersagen in ein Dictionary
prediction_results = {
    "method": predicted_method,
    "endpoint": predicted_endpoint,
    "presentation": predicted_presentation,
    "required": list(predicted_required),
    "select": list(predicted_select),
    "filter": list(predicted_filter),
    "expand": list(predicted_expand),
    "expand_select": list(predicted_expand_select)
}

# Umwandlung des Dictionaries in einen JSON-String
json_results = json.dumps(prediction_results, indent=4)

print("Prompt:", USER_PROMPT)

# Ausgabe der JSON-String
print(json_results)

Prompt: Hat der Auftrag 4711 einen Wareneingang?
{
    "method": "GET",
    "endpoint": "API_PRODUCTION_ORDER_2_SRV/A_ProductionOrder_2",
    "presentation": "blank",
    "required": [
        "ManufacturingOrder"
    ],
    "select": [
        "ManufacturingOrder",
        "MfgOrderConfirmedYieldQty",
        "OrderIsDelivered",
        "OrderIsPartiallyDelivered",
        "TotalQuantity"
    ],
    "filter": [],
    "expand": [],
    "expand_select": []
}
